In [1]:
import pandas as pd
import numpy  as np

   
import pickle
   

from scipy.sparse import issparse

 
from pathlib import Path


pd.set_option('display.max_columns', 5)

In [2]:
#### user specified
data_folder = "D:/analyze_Pearson_residuals/"

data_subfolder = "lupus"

data_path = Path ( data_folder + data_subfolder )

In [3]:
# output data
results_pkl = "Lg_complementary_samples.pkl"

# input data
counts_pkl = "counts_sparse.pkl" # sparse pandas data frame: cells in columns; genes in rows
gene_array_pkl  = "gene_array_complementary_samples.pkl"
df_cells_pkl  = "df_cells_complementary_samples.pkl" 


# path: output data
results_dsn = data_path / results_pkl

# paths: input data
counts_dsn = data_path / counts_pkl  
df_cells_dsn = data_path / df_cells_pkl
gene_array_dsn = data_path / gene_array_pkl

In [4]:
# https://github.com/scverse/scanpy/tree/master/scanpy/experimental/pp
# from   _normalization.py  rows 26-64

def _pearson_residuals(X, theta, clip, copy=False):    # removed check_values from parm list

    X = X.copy() if copy else X

    # check theta
    if theta <= 0:
        # TODO: would "underdispersion" with negative theta make sense?
        # then only theta=0 were undefined..
        raise ValueError('Pearson residuals require theta > 0')
    # prepare clipping
    if clip is None:
        n = X.shape[0]
        clip = np.sqrt(n)
    if clip < 0:
        raise ValueError("Pearson residuals require `clip>=0` or `clip=None`.")

# vfk 2022 08 16		
    # if check_values and not check_nonnegative_integers(X):
        # warn(
            # "`normalize_pearson_residuals()` expects raw count data, but non-integers were found.",
            # UserWarning,
        # )

    if issparse(X):
        sums_genes = np.sum(X, axis=0)
        sums_cells = np.sum(X, axis=1)
        sum_total = np.sum(sums_genes).squeeze()
    else:
        sums_genes = np.sum(X, axis=0, keepdims=True)
        sums_cells = np.sum(X, axis=1, keepdims=True)
        sum_total = np.sum(sums_genes)

    mu = np.array(sums_cells @ sums_genes / sum_total)
    diff = np.array(X - mu)
    residuals = diff / np.sqrt(mu + mu**2 / theta)

    # clip
    residuals = np.clip(residuals, a_min=-clip, a_max=clip)

    return residuals	
	
	
pctl_list = [.01,.05, .10, .25, .5, .75, .90, .95, .96, .97, .98, .99 ]

In [5]:
df_selected_cells = pd.read_pickle( df_cells_dsn )
print ( ' df_selected_cells:  ' )
print ( '\n', df_selected_cells )
print ( '\n\n df_selected_cells.sum:  ' )
print ( df_selected_cells.sum() )

f = open( gene_array_dsn, 'rb' )    
gene_select_array  = pickle.load(f)     
f.close()       
print ( '\n\n gene_select_array:  ' )
print ( gene_select_array )

 df_selected_cells:  

                       0      1
AAACATACAATGCC-1  False   True
AAACATACATTTCC-1   True  False
AAACATACCAGAAA-1   True  False
AAACATACCAGCTA-1   True  False
AAACATACCATGCA-1  False   True
...                 ...    ...
TTTGCATGCTAAGC-1   True  False
TTTGCATGGGACGA-1   True  False
TTTGCATGGTGAGG-1  False   True
TTTGCATGGTTTGG-1   True  False
TTTGCATGTCTTAC-1  False   True

[26820 rows x 2 columns]


 df_selected_cells.sum:  
0    13478
1    13342
dtype: int64


 gene_select_array:  
['ENSG00000244734' 'ENSG00000188536' 'ENSG00000167996' ...
 'ENSG00000241399' 'ENSG00000246100' 'ENSG00000158186']


In [6]:
df_counts_sparse = pd.read_pickle ( counts_dsn )
print ( ' df_counts_sparse:' )
print ( df_counts_sparse )
print ( '\n type( df_counts_sparse ) ', type ( df_counts_sparse ))
print ( '\n df_counts_sparse.sparse.density: ', df_counts_sparse.sparse.density)

 df_counts_sparse:
barcode          AAACATACAATGCC-1  AAACATACATTTCC-1  ...  TTTGCATGGTTTGG-1  \
ENSEMBL_ID                                           ...                     
ENSG00000239945                 0                 0  ...                 0   
ENSG00000237683                 0                 0  ...                 0   
ENSG00000228463                 0                 0  ...                 0   
ENSG00000228327                 0                 0  ...                 0   
ENSG00000237491                 0                 0  ...                 0   
...                           ...               ...  ...               ...   
ENSG00000160298                 0                 0  ...                 0   
ENSG00000160299                 0                 0  ...                 0   
ENSG00000160305                 0                 0  ...                 0   
ENSG00000160307                 0                 0  ...                 0   
ENSG00000160310                 0            

In [7]:
df_counts_sparse_all_samples_SD_positive = df_counts_sparse[ df_selected_cells.index ].loc [ gene_select_array ]
print ( ' df_counts_sparse_all_samples_SD_positive:' )
print ( df_counts_sparse_all_samples_SD_positive )
print ( '\n type( df_counts_sparse_all_samples_SD_positive ) ', type ( df_counts_sparse_all_samples_SD_positive ))
print ( '\n df_counts_sparse_all_samples_SD_positive.sparse.density: ', df_counts_sparse_all_samples_SD_positive.sparse.density)

del df_counts_sparse

 df_counts_sparse_all_samples_SD_positive:
barcode          AAACATACAATGCC-1  AAACATACATTTCC-1  ...  TTTGCATGGTTTGG-1  \
ENSEMBL_ID                                           ...                     
ENSG00000244734                 0                 0  ...                 0   
ENSG00000188536                 0                 0  ...                 0   
ENSG00000167996                32               388  ...                 8   
ENSG00000206172                 0                 0  ...                 0   
ENSG00000087086                 0               279  ...                 4   
...                           ...               ...  ...               ...   
ENSG00000113494                 0                 0  ...                 0   
ENSG00000196550                 0                 0  ...                 0   
ENSG00000241399                 0                 0  ...                 0   
ENSG00000246100                 0                 0  ...                 0   
ENSG00000158186      

In [8]:
df_clipped_half_cell_samples_list = []

for sample in  range(2):
  print ( '\n sample: ', sample )
  df_cell_sample = df_selected_cells[[sample]]
  df_cell_select = df_cell_sample [ df_cell_sample[sample] ]
  barcode_list = df_cell_select.index.values.tolist()
  
  df_counts_sparse_sample = df_counts_sparse_all_samples_SD_positive[ barcode_list ]     
  scipy_csr_mat = df_counts_sparse_sample.transpose().sparse.to_coo().tocsr(). astype ( 'float64'  )

  residuals = _pearson_residuals ( scipy_csr_mat, 100, None )
  residual_variance_clipped = np.var( residuals, axis=0)

  df_clipped_half_cell_sample = pd.DataFrame ( index = df_counts_sparse_all_samples_SD_positive.index, data = { sample: residual_variance_clipped  } ) 
  df_clipped_half_cell_sample.sort_values ( [sample], ascending=False, inplace=True )  

  df_clipped_half_cell_samples_list.append ( df_clipped_half_cell_sample )
  
  
df_clipped_half_cell_samples = pd.concat ( df_clipped_half_cell_samples_list, axis=1 )
print ( '\n df_clipped_half_cell_samples' )
print ( df_clipped_half_cell_samples  )
print ( '\n\n df_clipped_half_cell_samples.describe' )
print ( df_clipped_half_cell_samples.describe ( percentiles=pctl_list ) )


 sample:  0

 sample:  1

 df_clipped_half_cell_samples
                         0          1
ENSEMBL_ID                           
ENSG00000087086  50.711936  51.099089
ENSG00000167996  42.481003  42.322134
ENSG00000169245  40.903462  40.700295
ENSG00000187608  38.979586  38.098613
ENSG00000244734  37.104743  46.446606
...                    ...        ...
ENSG00000179331   0.679744   0.886626
ENSG00000079215   0.677357   0.889302
ENSG00000116991   0.665157   0.758688
ENSG00000241399   0.650771   0.716066
ENSG00000113494   0.636110   0.653710

[9091 rows x 2 columns]


 df_clipped_half_cell_samples.describe
                 0            1
count  9091.000000  9091.000000
mean      1.391172     1.398935
std       1.446612     1.507506
min       0.636110     0.622665
1%        0.786539     0.794902
5%        0.937915     0.931734
10%       1.022896     1.021301
25%       1.134747     1.135424
50%       1.240311     1.240975
75%       1.368792     1.374466
90%       1.619255     1.631509

In [9]:
df_clipped_half_cell_samples.to_pickle ( results_dsn )